In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

# Read the data
iowa_file_path = '/kaggle/input/home-data-for-ml-course/train.csv'
home_data = pd.read_csv(iowa_file_path)

# Remove rows with missing target
home_data.dropna(axis=0, subset=['SalePrice'], inplace=True) 

# Set the prediction target and separate it from the rest of the data
y = home_data.SalePrice
home_data.drop(['SalePrice'], axis=1, inplace=True) 

# Set the prediction data to X_full
X_full = home_data
display(X_full.columns)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


# Trying Descision Tree Regressor

In [3]:
# List of chosen features
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']

# Select data and display it
X1 = home_data[features]
display(X1.head())

# Split data into validation and training data
X1_train, X1_valid, y1_train, y1_valid = train_test_split(X1, y, random_state = 1)


,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
0,8450,2003,856,854,2,3,8
1,9600,1976,1262,0,2,3,6
2,11250,2001,920,866,2,3,6
3,9550,1915,961,756,1,3,7
4,14260,2000,1145,1053,2,4,9


In [4]:
# Defining Decision Tree
dt_model = DecisionTreeRegressor(random_state = 0)

# Fitting model
dt_model.fit(X1_train, y1_train)

# Get Predictions
dt_predictions = dt_model.predict(X1_valid)

# Print Mean Absolute Error
dt_val_mae = mean_absolute_error(dt_predictions, y1_valid)
print("Validation MAE for Decision Tree Regressor: ", dt_val_mae)


Validation MAE for Decision Tree Regressor:  29478.638356164385


**Finding the optimal number of leaf nodes to improve model accuracy**

In [5]:
# Function that will input the maximum number of leaf nodes and return the MAE
def get_mae(max_leaf_nodes, X_train, X_valid, y_train, y_valid):
    model = DecisionTreeRegressor(max_leaf_nodes = max_leaf_nodes, random_state = 0)
    model.fit(X_train, y_train)
    preds_val = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, preds_val)
    return(mae)

In [6]:
# For loop that prints MAE for a given list of nodes
for max_leaf_nodes in [5, 50, 500, 5000]:
    mae = get_mae(max_leaf_nodes, X1_train, X1_valid, y1_train, y1_valid)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, mae))

Max leaf nodes: 5  		 Mean Absolute Error:  35044
Max leaf nodes: 50  		 Mean Absolute Error:  27405
Max leaf nodes: 500  		 Mean Absolute Error:  29454
Max leaf nodes: 5000  		 Mean Absolute Error:  30162


# Trying Random Forest Regressor

In [7]:
# Define Random Forest Regressor Model
rf_model = RandomForestRegressor(random_state=1)

# Fit the model
rf_model.fit(X1_train, y1_train)

# Get predictions
rf_predictions = rf_model.predict(X1_valid)

# Print MAE
rf_val_mae = mean_absolute_error(rf_predictions, y1_valid)
print("Validation MAE for Random Forest Model: ", rf_val_mae)

Validation MAE for Random Forest Model:  21857.15912981083


# Adding More Features to Random Forest Model

In [8]:
# List of columns that do not have missing values and are numerical
features2 = features + ['MSSubClass','LotArea','OverallQual','OverallCond','YearBuilt','YearRemodAdd','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','FullBath',
'HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','MoSold','YrSold']

# Select data and display it
X2 = home_data[features2]
display(X2.head())

# Split data into validation and training sets
X2_train, X2_valid, y2_train, y2_valid = train_test_split(X2, y, random_state = 1)

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd,MSSubClass,LotArea,OverallQual,...,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,8450,2003,856,854,2,3,8,60,8450,7,...,0,0,61,0,0,0,0,0,2,2008
1,9600,1976,1262,0,2,3,6,20,9600,6,...,1,298,0,0,0,0,0,0,5,2007
2,11250,2001,920,866,2,3,6,60,11250,7,...,1,0,42,0,0,0,0,0,9,2008
3,9550,1915,961,756,1,3,7,70,9550,7,...,1,0,35,272,0,0,0,0,2,2006
4,14260,2000,1145,1053,2,4,9,60,14260,8,...,1,192,84,0,0,0,0,0,12,2008


In [9]:
# Define Random Forest Regressor
rf_model_2 = RandomForestRegressor(random_state=1)

# Fit the training data
rf_model_2.fit(X2_train, y2_train)

# Get predictions
rf_predictions_2 = rf_model_2.predict(X2_valid)

# Calculate and print MAE
rf_val_mae_2 = mean_absolute_error(rf_predictions_2, y2_valid)
print("Validation MAE for Random Forest Model: ", rf_val_mae_2)

Validation MAE for Random Forest Model:  17879.090684931507


# Trying Every Numerical Column

Getting Only Numerical Columns

In [10]:
# Selecting only numerical columns and displaying it
X3 = X_full.select_dtypes(exclude=['object'])
display(X3.head())

# Split data into training and validation data
X3_train, X3_valid, y3_train, y3_valid = train_test_split(X3, y, train_size=.8, test_size=.2, random_state = 0)


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,548,0,61,0,0,0,0,0,2,2008
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,460,298,0,0,0,0,0,0,5,2007
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,608,0,42,0,0,0,0,0,9,2008
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,642,0,35,272,0,0,0,0,2,2006
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,836,192,84,0,0,0,0,0,12,2008


Creating a scoring function

In [11]:
# Function that will output the MAE when giving validation and training data
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

Dropping Columns with Missing Values

In [12]:
# List of numerical columns missing data
cols_with_missing = [col for col in X3.columns 
                    if X3[col].isnull().any()]

# Selecting all numerical columns without any missing data
reduced_X3_train = X3_train.drop(cols_with_missing, axis=1)
reduced_X3_valid = X3_valid.drop(cols_with_missing, axis=1)

# Scoring dataset
print("MAE: ", score_dataset(reduced_X3_train, reduced_X3_valid, y3_train, y3_valid))

MAE:  17952.591404109586


Trying Imputation

In [13]:
# Defining Imputer
my_imputer = SimpleImputer(strategy = 'median')

# Imputing Data
imputed_X3_train = pd.DataFrame(my_imputer.fit_transform(X3_train))
imputed_X3_valid = pd.DataFrame(my_imputer.transform(X3_valid)) 


# Imputation removed the column names so putting them back
imputed_X3_train.columns = X3_train.columns
imputed_X3_valid.columns = X3_valid.columns

# Scoring dataset
print("MAE: ", score_dataset(imputed_X3_train, imputed_X3_valid, y3_train, y3_valid))

MAE:  18103.602945205483


# Using Categorical Variables

Oridinal Encoding

In [14]:
# Selecting and printing columns with missing values
cols_with_missing = [col for col in X_full.columns if X_full[col].isnull().any()]
print(cols_with_missing)

# Dropping columns with missing values
X4 = X_full.copy().drop(cols_with_missing, axis=1)

# Splitting validation and training data
X4_train, X4_valid, y4_train, y4_valid = train_test_split(X4, y, train_size=0.8, test_size=0.2, random_state=0)

# Categorical columns
object_cols = [col for col in X4_train.columns if X4_train[col].dtype == 'object']

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if
                  set(X4_valid[col]).issubset(set(X4_train[col]))]

# Problematic columns 
bad_label_cols = list(set(object_cols)-set(good_label_cols))

['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']


In [15]:
# Dropping problematic Columns
label_X4_train = X4_train.drop(bad_label_cols, axis=1)
label_X4_valid = X4_valid.drop(bad_label_cols, axis=1)

# Defining Ordinal Encoder
ordinal_encoder = OrdinalEncoder()

# Applying Ordinal Encoder to categorical data that is not problematic
label_X4_train[good_label_cols] = ordinal_encoder.fit_transform(X4_train[good_label_cols])
label_X4_valid[good_label_cols] = ordinal_encoder.transform(X4_valid[good_label_cols])

# Scoring dataset
print('MAE: ', score_dataset(label_X4_train, label_X4_valid, y4_train, y4_valid))

MAE:  17251.037739726027


Trying a One-Hot-Encoder

In [16]:
# Finding low and high cardinality colmns.
# Low cardinality columns were arbitrarilty chosen to have less than ten values 
low_cardinality_cols = [col for col in object_cols if X4_train[col].nunique()<10]
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))
print (high_cardinality_cols)

['Neighborhood', 'Exterior1st', 'Exterior2nd']


In [17]:
# Defining One Hot Encoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse = False)

# Applying Encoder to each column with low cardinality
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X4_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X4_valid[low_cardinality_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X4_train.index
OH_cols_valid.index = X4_valid.index

# Remove categorical columns
num_X4_train = X4_train.drop(object_cols, axis=1)
num_X4_valid = X4_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X4_train = pd.concat([num_X4_train, OH_cols_train], axis=1)
OH_X4_valid = pd.concat([num_X4_valid, OH_cols_valid], axis=1)

# Ensure all columns have string Type
OH_X4_train.columns = OH_X4_train.columns.astype(str)
OH_X4_valid.columns = OH_X4_valid.columns.astype(str)

# Score dataset
print("MAE: ", score_dataset(OH_X4_train, OH_X4_valid, y4_train, y4_valid))

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


MAE:  17514.224246575344


# Pipeline Implementation

In [18]:
# Select all predictor columns
X5 = X_full.copy()

# Split into validation and training data
X5_train, X5_valid, y5_train, y5_valid = train_test_split(X5, y, train_size=0.8, test_size=0.2, random_state=0)

# Categorical columns in the data
all_object_cols_X5 = [col for col in X5.columns if X5[col].dtype == 'object']

# Categorical columns that can be safely Ordinal Encoded
categorical_cols_X5 = [col for col in all_object_cols_X5 if
                  set(X5_valid[col]).issubset(set(X5_train[col]))]

# Selecting and displaying problematic categorical columns 
bad_label_cols_X5 = list(set(all_object_cols_X5)-set(categorical_cols_X5))
display(bad_label_cols_X5)

# Dropping problematic columns
X5_train.drop(bad_label_cols_X5, axis=1, inplace=True)
X5_valid.drop(bad_label_cols_X5, axis=1, inplace=True) 

# Numerical Columns
numerical_cols_X5 = [col for col in X5.columns if 
                 X5[col].dtype in ['int64', 'float64']]

['MiscFeature', 'Condition2', 'RoofMatl', 'Functional']

In [19]:
# Preprocessing for numerical data
# Simple Imputer will impute the average value to each missing variable
numerical_transformer = SimpleImputer(strategy='mean') 

# Preprocessing for categorical data
# Simple Imputer will give a value to every missing variable
# Ordinal Encoder will be able to handle unknown values found in the validation set, allowing for cross-validation later
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant')),
                                         ('Ordinal encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))]) 

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols_X5),
    ('cat', categorical_transformer, categorical_cols_X5)
])

# Bundle preprocessing and a Random Forest Regressor model in a pipeline
pipeline_1 = Pipeline(steps=[('preprocessor', preprocessor),
                             ('model', RandomForestRegressor(random_state=1))
                             ])

# Preprocessing of training data, fit model
pipeline_1.fit(X5_train, y5_train)

# Get Predictions
predictions = pipeline_1.predict(X5_valid)

# Print MAE
print('MAE: ', mean_absolute_error(y5_valid, predictions))

MAE:  17087.34640410959


# Tweaking the Random Forest Regressor

In [20]:
# Function to return the average MAE over 3 CV folds of a Random Forest model for a given number of n_estimator
def get_score(n_estimators):
    
    # Create new pipeline with n_estimators as a variable
    pipeline_2 = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(n_estimators=n_estimators, random_state=1))
    ])
    
    # Get list of MAE using model 
    scores = -1 * cross_val_score(pipeline_2, X5, y,
                                 cv = 3,
                                 scoring = 'neg_mean_absolute_error') 

    # Return the average MAE
    return scores.mean()

In [21]:
import matplotlib.pyplot as plt
%matplotlib inline

# Create Dictionary
results = {}


'''# For loop stores n_estimators in intervals of 50 from 50 - 400 and the MAE as keys and values respectively 
for i in range(1,9):
    results[i*50] = get_score(i*50)

# Prints a graph plotting MAE vs. n_estimators in a Random Forest Regressor Model
plt.plot(list(results.keys()), list(results.values()))
plt.show()'''


'# For loop stores n_estimators in intervals of 50 from 50 - 400 and the MAE as keys and values respectively \nfor i in range(1,9):\n    results[i*50] = get_score(i*50)\n\n# Prints a graph plotting MAE vs. n_estimators in a Random Forest Regressor Model\nplt.plot(list(results.keys()), list(results.values()))\nplt.show()'

# Trying XGBoost

In [22]:
# Define the model 
XGB_model = XGBRegressor(n_estimators = 500, 
                           learning_rate=0.05,
                        )

# Create new pipeline with previous preprocessor and new model
pipeline_3 = Pipeline(steps=[
                        ('preprocessor', preprocessor),
                        ('model', XGB_model)
])

# Fit the model
pipeline_3.fit(X5_train, y5_train)

# Get predictions
XGB_preds = pipeline_3.predict(X5_valid)

# Print MAE
print("MAE: ", mean_absolute_error(y5_valid, XGB_preds))

MAE:  17042.19630244007


Adding hyperparamaters to XGBoost. Need to create another pipeline that will only preprocess the data, because pipeline_3 doesn't allow for early_stopping. Also note that pipeline_1 did not process all the categorical columns as X5_train dropped the bad columns. This new preprocessor will be able to process all columns. This is necessary for when we apply the entire data set to the model, we will be able to process every column without dropping necessary info. 

In [23]:
# Select all predictor columns
X6 = X_full.copy()

# Split into validation and training data
X6_train, X6_valid, y6_train, y6_valid = train_test_split(X6, y, train_size = .8, test_size=.2, random_state = 0)

# All categorical columns
categorical_cols_X6 = [col for col in X6.columns
                if X6[col].dtype == 'object']

# All numerical columns
numerical_cols_X6 = [col for col in X6.columns
                if X6[col].dtype in ['int64', 'float64']]

# Preprocessing for numerical columns
numerical_transformer_2 = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

# Preprocessing for categorical columns
categorical_transformer_2 = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
    #('onehot', OneHotEncoder(handle_unknown='ignore', sparse = False))
])

# Bundle preprocessing 
preprocessor_2 = ColumnTransformer(transformers=[
    ('numerical_transformer', numerical_transformer_2, numerical_cols_X6),
    ('categorical_transformer', categorical_transformer_2, categorical_cols_X6)
])

# Create pipeline for preprocessing
preprocessing_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_2)
])

# Preprocess validation and training data
X6_train_processed = preprocessing_pipeline.fit_transform(X6_train)
X6_valid_processed = preprocessing_pipeline.transform(X6_valid)


In [24]:
# Define model
XGB_model_2 = XGBRegressor(n_estimators = 500, 
                           learning_rate=0.05,
                           )

# Fit the model with early stopping rounds 
XGB_model_2.fit(X6_train_processed, y6_train,
               early_stopping_rounds=20,
               eval_set=[(X6_valid_processed, y6_valid)],
               verbose=False)

# Get predictions
XGB_model_2_pred = XGB_model_2.predict(X6_valid_processed)

# Print MAE
print("MAE: ", mean_absolute_error(y6_valid, XGB_model_2_pred))

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


MAE:  16904.24075610017


**Hypertuning Paramaters**

In [25]:
"""# Set paramater distributions
param_grid = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 6, 9],
    'colsample_bytree': [0.5, 0.7, 1],
    'subsample': [0.7, 0.8, 1],
    'gamma': [0, 1, 5]
}

# Set up RandomizedSearchCV to find optimal paramaters
search = RandomizedSearchCV(XGB_model_2, 
                            param_distributions=param_grid, 
                            cv=3, 
                            n_iter=50, 
                            scoring='neg_mean_absolute_error', 
                            random_state=0)

# Fit the model
search.fit(X6_train_processed, y6_train)

# Print the results
print("Best parameters:", search.best_params_)"""

'# Set paramater distributions\nparam_grid = {\n    \'n_estimators\': [100, 200, 500],\n    \'learning_rate\': [0.01, 0.05, 0.1],\n    \'max_depth\': [3, 6, 9],\n    \'colsample_bytree\': [0.5, 0.7, 1],\n    \'subsample\': [0.7, 0.8, 1],\n    \'gamma\': [0, 1, 5]\n}\n\n# Set up RandomizedSearchCV to find optimal paramaters\nsearch = RandomizedSearchCV(XGB_model_2, \n                            param_distributions=param_grid, \n                            cv=3, \n                            n_iter=50, \n                            scoring=\'neg_mean_absolute_error\', \n                            random_state=0)\n\n# Fit the model\nsearch.fit(X6_train_processed, y6_train)\n\n# Print the results\nprint("Best parameters:", search.best_params_)'

In [26]:
# XGBoost model with optimal paramaters
XGB_model_3 = XGBRegressor(n_estimators = 500, 
                           learning_rate=0.05,
                           max_depth=3,
                           gamma=1,
                           colsample_bytree=0.7,
                           subsample=0.8)

# Fit the model
# Adding early stopping worsened scores
XGB_model_3.fit(X6_train_processed, y6_train)

# Get predictions
XGB_model_3_pred = XGB_model_3.predict(X6_valid_processed)

# Print MAE
print("MAE: ", mean_absolute_error(y6_valid, XGB_model_3_pred))



MAE:  15586.202790560788


# Final Submission

In [27]:
# Read test data
test_data_path = '/kaggle/input/home-data-for-ml-course/test.csv'
test_data = pd.read_csv(test_data_path)

# Preprocess all the prediction data and fit the pipeline to its entirety
processed_X6 = preprocessing_pipeline.fit_transform(X6)

# Preprocess the testing data 
processed_test_data = preprocessing_pipeline.transform(test_data)

# Final Model
final_model = XGBRegressor(n_estimators = 500, 
                           learning_rate=0.05,
                           max_depth=3,
                           gamma=1,
                           colsample_bytree=0.7,
                           subsample=0.8)

# Fit the model
final_model.fit(processed_X6, y)

# Get predictions from test data
test_preds = final_model.predict(processed_test_data)

# Output Predictions
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_preds})
output.to_csv('submission.csv', index=False)